In [2]:
# Import the modules
import cv2
from sklearn.externals import joblib
import numpy as np
from keras.models import load_model
file = "test_image.png"
def show_webcam():
    cam = cv2.VideoCapture(0)
    while True:
        ret_val, img = cam.read()
        cv2.imshow('my webcam', img)
        if cv2.waitKey(1) == 27: 
            cv2.imwrite(file , img) #capture image
            break  # esc to quit
    cv2.destroyAllWindows()

#show_webcam()
inp_pic = "test_image.png"
    # Load the Keras CNN trained model
model = load_model('CLASSIFIER.h5')

    # Original image
im = cv2.imread(inp_pic)

    ################# OLD Algorithm with Fixed Threshold #####################
    # Convert to grayscale and apply Gaussian filtering
    # im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
    # Threshold the image
    # ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)
    # cv2.imshow("Thres",im_th)
    ###########################################################################


    ################# NEW Algorithm with Adaptive Threshold #########################################
    # Read image in grayscale mode
img = cv2.imread(inp_pic,0)
    
    # Median Blur and Gaussian Blur to remove Noise
img = cv2.medianBlur(img,3)
img = cv2.GaussianBlur(img, (5, 5), 0)
    # Adaptive Threshold for handling lightning
im_th = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,5)
kernel = np.ones((1,1),np.uint8)
im_th = cv2.dilate(im_th,kernel,iterations = 4)

    ##################################################################################################


    # Find contours in the image
_ ,ctrs, hier= cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#print(ctrs.shape)
   # Get rectangles contains each contour
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
#print(rects)
    # For each rectangular region, predict using cnn model
for rect in rects:
        # Draw the rectangles
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
        # Make the rectangular region around the digit
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    #print(leng)
        # Resize the image
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi2 = roi.reshape(1,28,28,1)
  
     # Input for CNN Model
    roi = roi[np.newaxis,np.newaxis,:,:]
    #print(roi.shape)

        #Input for Feed Forward Model
    #roi = roi.flatten()
    #roi = roi[np.newaxis]
    nbr = model.predict_classes(roi2,verbose=0)
    
    cv2.putText(im, str(int(nbr[0])), (int(rect[0]), int(rect[1])),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)

    cv2.imshow("Resulting Image with Predicted numbers", im)
    cv2.waitKey()

cv2.destroyAllWindows()